In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",1,0.571645,"""a""",0.737477,0.656159,0.416856,0.517078,0.204496,-0.31775,0.349539
0.841471,1,"""a""",0,0.282856,"""a""",0.443798,0.207286,0.393704,0.511206,0.827261,-0.461779,0.111684
0.909297,2,"""a""",1,0.138441,"""a""",0.033054,0.161957,0.898019,0.618572,0.818938,-1.293453,-0.012571
0.14112,3,"""a""",1,0.981857,"""a""",0.358035,0.121584,0.982338,0.794143,0.955313,-1.383307,0.003056
-0.756802,4,"""a""",1,0.177977,"""a""",0.024048,0.071361,0.34734,0.083007,0.602038,-0.495975,0.000526


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.517078,-0.204496
-0.841471,-0.511206,-0.827261
-0.909297,-0.618572,-0.818938
-0.14112,-0.794143,-0.955313
0.756802,0.434071,-0.397542


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.493905, -0.335693]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.493905, -0.335693]","[0.087113, 0.407661]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.493905, -0.335693]","[0.087113, 0.407661]"


In [10]:
pds.config.LIN_REG_EXPR_F64 = True

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.220143,0.001668,131.962818,0.0,0.216873,0.223413,0.994316,0.994314
"""exp(x2)""",0.176087,0.000671,262.530182,0.0,0.174772,0.177402,0.994316,0.994314
"""sin(x3)""",-1.741329,0.001348,-1292.128026,0.0,-1.74397,-1.738687,0.994316,0.994314
"""__bias__""",-0.111669,0.001484,-75.237283,0.0,-0.114579,-0.10876,0.994316,0.994314


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.220145,0.001668,131.964279,0.0,0.216875,0.223415,0.994316,0.994314
"""exp(x2)""",0.176086,0.000671,262.528442,0.0,0.174771,0.177401,0.994316,0.994314
"""sin(x3)""",-1.74133,0.001348,-1292.129395,0.0,-1.743972,-1.738689,0.994316,0.994314
"""__bias__""",-0.111669,0.001484,-75.23671,0.0,-0.114578,-0.108759,0.994316,0.994314


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.472364, -0.355623]"
"""a""","[-0.472364, -0.355623]"
"""a""","[-0.472364, -0.355623]"
"""a""","[-0.472364, -0.355623]"
"""a""","[-0.472364, -0.355623]"
…,…
"""b""","[-0.515457, -0.315614]"
"""b""","[-0.515457, -0.315614]"
"""b""","[-0.515457, -0.315614]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.737477,0.656159,-0.31775,-0.584511,0.266761
0.443798,0.207286,-0.461779,-0.288779,-0.173
0.033054,0.161957,-1.293453,-0.070693,-1.22276
0.358035,0.121584,-1.383307,-0.217651,-1.165656
0.024048,0.071361,-0.495975,-0.035833,-0.460142


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.472364, -0.355623]"
"""b""","[-0.515457, -0.315614]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.303934, -0.179282]"
"""b""","[-0.35244, -0.13801]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""b""",-0.536791
"""a""",-0.539748


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.31775,0.737477,0.656159,null,null
-0.461779,0.443798,0.207286,null,null
-1.293453,0.033054,0.161957,null,null
-1.383307,0.358035,0.121584,null,null
-0.495975,0.024048,0.071361,"[-1.07871, -0.079815]",-0.031637
…,…,…,…,…
0.133152,0.87549,0.062996,"[0.257336, -1.217235]",0.148614
-0.850142,0.312835,0.931315,"[0.334055, -1.284107]",-1.091404
-0.967542,0.010876,0.44341,"[0.238999, -1.425125]",-0.629315


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.191436, 28.869868, 28.727088]"


In [22]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.962459,"[0.967373, -0.253355]"
28.732768,"[0.253355, 0.967373]"


In [23]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
0.089785
-0.073676
0.032295
0.167587
-0.430843


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.337436,1.008287,0.484528,0.480614,0.482563,0.485088,0.488582
"""a""",0.340379,1.027516,0.494351,0.500197,0.497257,0.498456,0.485221


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [27]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [28]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [29]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [30]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [31]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [32]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [33]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [34]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
0.722458
-0.484456
0.624449


In [35]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,2.537438,null
null,0.706374,null
0.722458,-1.12339,1.779559
-0.484456,1.175272,0.614123
0.624449,2.0061,1.090509


In [36]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""WfNF""",null
null,"""1U""",null
0.722458,"""Dc""","""m4G"""
-0.484456,"""TJX7""","""R"""
0.624449,"""M1p4k""","""upGxx"""


In [37]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
0.722458,"""qNl62"""
-0.484456,"""FmlOU"""
0.624449,"""H7pIj"""


In [38]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,2.036459,null,2.036278
null,0.504016,null,0.503723
0.722458,-0.128867,2.427995,-0.128661
-0.484456,0.010985,0.599322,0.01106
0.624449,0.223828,0.851255,0.223886


In [39]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.088596,-0.183458,7
null,0.172991,1.556938,1
0.722458,0.322467,-1.923161,1
-0.484456,0.882868,1.468614,1
0.624449,0.080384,0.374051,3


In [40]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
1.043495,0.296886,0.421531,0.809964


In [41]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.985925,0.326689,4,8
1,0.63306,0.62686,1,8
2,0.380307,0.216754,1,7
0,0.272666,0.590648,1,3
1,0.157388,0.165123,4,7


In [42]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.529576,0.596417}","{37.30612,0.408836}","{0.304024,0.875426}"


In [43]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬─────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test              ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                 ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]           ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪═════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.488935,0.62492}  ┆ {27.841443,0.832804} ┆ {1.319094,0.260586} │
│ 1         ┆ {0.051561,0.958881} ┆ {37.289702,0.409569} ┆ {1.031595,0.389518} │
│ 2         ┆ {0.373462,0.708828} ┆ {31.827997,0.667352} ┆ {0.931224,0.444795} │
└───────────┴─────────────────────┴──────────────────────┴─────────────────────┘


In [44]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
536,0.1072
562,0.1124
524,0.1048
555,0.111
565,0.113
561,0.1122
550,0.11
549,0.1098
598,0.1196


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [45]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [46]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.20653,0.081537,0.810793,0.656258,2.087901,16
1,0.041573,0.209563,0.208995,0.480468,0.440177,11
2,0.752516,0.703126,0.407974,0.16779,1.845556,24
3,0.264682,0.219408,0.927873,0.927017,8.944035,11
4,0.054341,0.90091,0.814839,0.971407,9.390814,13


In [47]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.20653,0.081537,0.810793,0.656258,2.087901,320
1,0.041573,0.209563,0.208995,0.480468,0.440177,157
2,0.752516,0.703126,0.407974,0.16779,1.845556,20
3,0.264682,0.219408,0.927873,0.927017,8.944035,749
4,0.054341,0.90091,0.814839,0.971407,9.390814,645


In [48]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.20653,0.081537,0.810793,0.656258,2.087901,"[0, 1621, … 868]"
1,0.041573,0.209563,0.208995,0.480468,0.440177,"[1, 760, … 695]"
2,0.752516,0.703126,0.407974,0.16779,1.845556,"[2, 460, … 775]"
3,0.264682,0.219408,0.927873,0.927017,8.944035,"[3, 946, … 448]"
4,0.054341,0.90091,0.814839,0.971407,9.390814,"[4, 1176, … 989]"


In [49]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 1621, … 1819] ┆ 8                  │
│ 1   ┆ [1, 760, … 911]   ┆ 7                  │
│ 2   ┆ [2, 460, … 1725]  ┆ 12                 │
│ 3   ┆ [3, 946, … 592]   ┆ 6                  │
│ 4   ┆ [4, 1176, … 1874] ┆ 8                  │
└─────┴───────────────────┴────────────────────┘


In [50]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.20653,0.081537,0.810793,0.656258,2.087901,"[0, 1621, … 868]","[0.0, 0.042358, … 0.063033]"
1,0.041573,0.209563,0.208995,0.480468,0.440177,"[1, 760, … 695]","[0.0, 0.025515, … 0.062904]"
2,0.752516,0.703126,0.407974,0.16779,1.845556,"[2, 460, … 775]","[0.0, 0.047533, … 0.066362]"
3,0.264682,0.219408,0.927873,0.927017,8.944035,"[3, 946, … 448]","[0.0, 0.023069, … 0.058875]"
4,0.054341,0.90091,0.814839,0.971407,9.390814,"[4, 1176, … 989]","[0.0, 0.044666, … 0.053862]"


In [51]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
2,0.752516,0.703126,0.407974,0.16779,1.845556
5,0.483772,0.267881,0.170574,0.481354,0.444976
6,0.664823,0.145737,0.32259,0.38019,8.304767
7,0.091734,0.348827,0.438454,0.027811,5.119715
8,0.454813,0.652287,0.362894,0.020651,0.053776


In [52]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
43,0.560007,0.505178,0.875312,0.027995,0.555443
61,0.415983,0.468536,0.814105,0.453221,9.570076
73,0.484356,0.586594,0.030413,0.257265,2.617261
84,0.473782,0.543313,0.712958,0.040817,5.141623
91,0.459296,0.532573,0.909997,0.991585,0.741529


In [53]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
200,0.523137,0.523701,0.650535,0.889153,6.136111
205,0.511204,0.521639,0.097357,0.524813,3.90081
234,0.523151,0.497033,0.506257,0.92592,3.818622
280,0.525228,0.423796,0.372138,0.163537,9.241056
339,0.441779,0.510457,0.527714,0.705563,7.571939


In [54]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 1721, … 1535]",5
1,"[1, 71, … 1144]",5
2,"[2, 1138, … 1371]",5
3,"[3, 1797, 1923]",3
4,"[4, 620, … 76]",6


# Compatibility

## Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures. 

## Using PDS with Narwhals

Limited

In [55]:
import pandas as pd
import numpy as np
import polars as pl
import polars_ds as pds
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_14672/181413171.py:5: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.139499,2,1,"""Ns""","""p2"""
0.0,0.161041,2,2,"""M""","""Ro"""
0.0,0.68341,1,3,"""da""","""p"""
0.0,0.752049,2,7,"""V""","""U"""
0.0,0.115851,2,3,"""d""","""n6"""


In [56]:
df_pd = df.to_pandas()

In [57]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

0    0.3
Name: , dtype: float64

In [58]:
# Polars Series
print(pds2.query_roc_auc(df["actual"], df["predicted"]))
# NumPy
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_numpy(), df["predicted"].to_numpy()))
pds2.return_numpy = False
# Pandas
print(pds2.query_roc_auc(df["actual"].to_pandas(), df["predicted"].to_pandas()))
# PyArrow
# Arrow series can be inputs, but the output cannot be converted correctly. Please let me know if you have a fix.
# The work around is to use NumPy for Arrow
pds2.return_numpy = True
print(pds2.query_roc_auc(df["actual"].to_arrow(), df["predicted"].to_arrow()))
# Other array-protocal compatible inputs
# print(pds2.query_roc_auc(df["actual"].to_jax(), df["predicted"].to_jax()))

pds2.return_numpy = False


shape: (1,)
Series: '' [f64]
[
	0.499078
]
[0.49907755]
0    0.499078
Name: , dtype: float64
[0.49907755]


In [59]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
)


array([0.00776855])

In [60]:
df_pd = df.to_pandas()
df_pd["levenshtein_dist"] = pds2.str_leven(df_pd["s1"], df_pd["s2"])
df_pd.head()

,actual,predicted,0-2,0-9,s1,s2,levenshtein_dist
0,1.0,0.139499,2,1,Ns,p2,2
1,0.0,0.161041,2,2,M,Ro,2
2,0.0,0.683410,1,3,da,p,2
3,0.0,0.752049,2,7,V,U,1
4,0.0,0.115851,2,3,d,n6,2


In [61]:
# If you are using Narwhals, well, Narwhal expressions are not Polars expressions.
# Using the pds2 module, you can run pds functions in map_batches, but this is limited to 1 input column.

import narwhals as nw

df_nw = nw.from_native(df_pd)
df_nw.with_columns(
    nw_levenshtein_dist = nw.col("s1").map_batches(lambda s: pds2.str_leven(s.to_numpy(), pl.lit('k9')))
).head()

┌─────────────────────────────────────────────────────────────────────────────┐
|                             Narwhals DataFrame                              |
|-----------------------------------------------------------------------------|
|   actual  predicted  0-2  0-9  s1  s2  levenshtein_dist  nw_levenshtein_dist|
|0     1.0   0.139499    2    1  Ns  p2                 2                    2|
|1     0.0   0.161041    2    2   M  Ro                 2                    2|
|2     0.0   0.683410    1    3  da   p                 2                    2|
|3     0.0   0.752049    2    7   V   U                 1                    2|
|4     0.0   0.115851    2    3   d  n6                 2                    2|
└─────────────────────────────────────────────────────────────────────────────┘